# Lung Cancer

## Index <a id="Index"></a>

1. [Introduction](#Introduction)
    1. [Project Overview](#Project-Overview)
    2. [Objectives](#Objectives)
    3. [Our Approach](#Our-Approach)
2. [Imports](#Imports)
3. [Obtaining the features](#Obtain-the-features)
4. [Data Clean-up](#Data-Clean-up)
5. [Data Analysis](#Data-Analysis)
6. [Classification](#Classification)
    1. [Decision Tree Classifier](#Decision-Tree-Classifier)
    2. [Random Forest Classifier](#Random-Forest-Classifier)
    3. [Gradient Boosting Classifier](#Gradient-Boosting-Classifier)
    4. [K Neighbors Classifier](#Knn)
    5. [Support Vector Machines](#Support-Vector-Machines)
    6. [Naive Bayes](#Naive-Bayes)
        1. [GaussianNB](#GaussianNB)
        2. [MultinomialNB](#MultinomialNB)
7. [Conclusion](#Conclusion)

## Introduction <a id="Introduction"></a>
[Back to index](#Index)

### Project Overview <a id="Project-Overview"></a>

In this project our goal is to develop a  Data Science-based solution to solve the problem of Lung Cancer
Classification using Computerized Tomography (CT) Data. Our initial data comes from images of CT scans from 1010 patients and we intend to extract its features from the images.

### Objectives <a id="Objectives"></a>

Our goal is to develop a robust and accurate classification system that can distinguish between benign and malignant nodules, which could assist healthcare professionals in making informed decisions, as well as improve our programming skills and machine learning and data science knowledge.

### Our Approach <a id="Our-Approach"></a>

The first step is to extract the features from the CT images and colect them in a csv.
Next we will evaluate the need for cleaning the data and preprocess it.
After that, the final step is to develop machine learning models and avaluate its performance.

## Imports
[Back to index](#Index)

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

### Obtaining the features <a id="Obtain-the-features"></a>
[Back to index](#Index)

Running the python script:

`python get_features.py`
 
we are going to acess the information in the pylidc library and extract its features and calculating the mean or mode for all annotators as well as extracting all radiomics features

%run get_features.py

To run the script change the cell above from markdown to code and run it.

If you interrupt the cell above without it finishing you need to restart your kerner to prevent: 

`PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)`





## Data Clean-up <a id="Data-Clean-up"></a>
[Back to index](#Index)

Now we are going to clean the data

For starters let's read the csv and print the features dataset

In [ ]:
df = pd.read_csv('radiomic_features.csv')
df

Next, we are going to check if there are any null values

In [ ]:
df.isna().sum()

Since there are no null values we don't need to perform any null value handling

Now we are going to check if there are columns with only one value

In [ ]:
df.nunique().tolist()

A lot of columns have only one value, and we can conclude that they will not be helpful at predicting the label. So, we are going to drop them.

In [ ]:
unique_value_counts = df.nunique()
columns_with_single_unique_value = unique_value_counts[unique_value_counts == 1].index
df.drop(columns=columns_with_single_unique_value,inplace=True)

Let's see the updated dataset

In [ ]:
df

As we can see, in the dataset some columns have Ids and tuples with numbers and since there is no point in label encoding them since these columns don´t really represent features of the nodules and just information about the general images or masks. So, we will drop those columns, remaining just numerical data.

In [ ]:
df = df.select_dtypes(include=[int, float])
df

Since we applied the mean to all of the values in the annotations for each nodule they are in between [1,5]. We are going to check the count of each value in the malignancy column.

In [ ]:
df["malignancy"].value_counts()

As we can see, there are a lot of values with 3, so we are going to drop them since they are not useful for our classification.
The rest of the values are going to be converted to 0 or 1, where 0 is benign and 1 is malignant.

In [ ]:
df = df[df['malignancy'] != 3]

df.loc[:, "malignancy"] = df["malignancy"].apply(lambda x: 1 if x > 3 else 0)

Let's see the updated malignancy column

In [ ]:
df["malignancy"].value_counts()

As shown above, the labels are not balanced, so we are going to use oversampling to balance them.

In [ ]:
smote = SMOTE(sampling_strategy='auto', random_state=42)
df, _ = smote.fit_resample(df, df['malignancy'])

Now the values should be equally distributed

In [ ]:
df["malignancy"].value_counts()


Afterwards we need to normalize the columns using Min-Max scaling due to not having scale consistency

In [ ]:
df = (df - df.min()) / (df.max() - df.min())

Since the column names are very long and not concise nor clear we will simplify them to make them more understandable.

In [ ]:
new_column_names = [
    'Mean',
    'VoxelNum',
    'VolumeNum',
    'Elongation',
    'Flatness',
    'LeastAxisLength',
    'MajorAxisLength',
    'DiameterColumn',
    'DiameterRow',
    'DiameterSlice',
    'Max3DDiameter',
    'MeshVolume',
    'MinorAxisLength',
    'Sphericity',
    'SurfaceArea',
    'SurfaceVolRatio',
    'VoxelVol',
    'Energy',
    'TotalEnergy',
    'DiffEntropy',
    'JointEntropy',
    'SumEntropy',
    'DependEntropy',
    'DependNonUniformity',
    'DependNonUniformityNorm',
    'DependVariance',
    'GrayLevelNonUniformity',
    'LargeDependEmphasis',
    'LargeDependHighGLEmphasis',
    'LargeDependLowGLEmphasis',
    'SmallDependEmphasis',
    'SmallDependHighGLEmphasis',
    'SmallDependLowGLEmphasis',
    'GLNonUniformity',
    'LongRunEmphasis',
    'LongRunHighGLEmphasis',
    'LongRunLowGLEmphasis',
    'RunEntropy',
    'RunLengthNonUniformity',
    'RunLenNonUniformityNorm',
    'RunPercentage',
    'RunVariance',
    'ShortRunEmphasis',
    'ShortRunHighGLEmphasis',
    'ShortRunLowGLEmphasis',
    'GLNonUniformity_GLSZM',
    'LargeAreaEmphasis',
    'LargeAreaHighGLEmphasis',
    'LargeAreaLowGLEmphasis',
    'SizeZoneNonUniformity',
    'SizeZoneNonUniformityNorm',
    'SmallAreaEmphasis',
    'SmallAreaHighGLEmphasis',
    'SmallAreaLowGLEmphasis',
    'ZoneEntropy',
    'ZonePercentage',
    'ZoneVariance',
    'Subtlety',
    'InternalStructure',
    'Sphericity',
    'Margin',
    'Lobulation',
    'Spiculation',
    'Texture',
    'Malignancy'
]
df.columns = new_column_names

Now let's take a look at the updated dataset

In [ ]:
df

## Data Analysis <a id="Data-Analysis"></a>
[Back to index](#Index)

Rigth now we are going to analyze the data and see if there are any correlations between the features and the label in order to have better performance in the our classification models.

### Correlation Analysis

In [ ]:
df.select_dtypes(include=['object'])
numeric_columns = df.select_dtypes(include=['object'])
correlation_matrix = numeric_columns.corr()


This code provides insights into the distribution and characteristics of these diagnostic values, wich may represent various image and mask features 

In [ ]:
# Diagnostics Columns
mean_image = df['Mean']
voxel_num = df['VoxelNum']
volume_num = df['VolumeNum']

# Summary statistics
print(mean_image.describe())
print(voxel_num.describe())
print(volume_num.describe())

This code segment focuses on a set of columns related to shape features of tumors

In [ ]:
# shape features
shape_columns = [
    'Elongation',
    'Flatness',
    'LeastAxisLength',
    'MajorAxisLength',
    'DiameterColumn',
    'DiameterRow',
    'DiameterSlice',
    'Max3DDiameter',
    'MeshVolume',
    'MinorAxisLength',
    'Sphericity',
    'SurfaceArea',
    'SurfaceVolRatio',
    'VoxelVol',
]

# Summary statistics for shape features
shape_summary = df[shape_columns].describe()
print(shape_summary)

# Correlation analysis for shape features
shape_correlation_matrix = df[shape_columns].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(shape_correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix for Shape Features")
plt.show()

In the correlation matrix displayed above, we observe a rich network of interconnections among various features. Notably, a prominent 5x5 square region stands out, marked in pink, indicating a substantial degree of correlation between the enclosed features. In practical terms, this means that when one feature experiences an increase, a corresponding increase in the other is highly likely, signifying a strong positive correlation.

In [ ]:
# GLSZM Features
glszm_columns = [
    'GLNonUniformity_GLSZM',
    'LargeAreaEmphasis',
    'LargeAreaHighGLEmphasis',
    'LargeAreaLowGLEmphasis',
    'SizeZoneNonUniformity',
    'SizeZoneNonUniformityNorm',
    'SmallAreaEmphasis',
    'SmallAreaHighGLEmphasis',
    'SmallAreaLowGLEmphasis',
    'ZoneEntropy',
    'ZonePercentage',
    'ZoneVariance',
]

# Summary statistics for GLSZM features
glszm_summary = df[glszm_columns].describe()
print(glszm_summary)

# Correlation analysis for GLSZM features
glszm_correlation_matrix = df[glszm_columns].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(glszm_correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix for GLSZM Features")
plt.show()

The Gray-Level Size Zone Matrix (GLSZM) is a quantitative image analysis technique used to characterize the spatial distribution of zones of specific gray levels within an image. It provides information about the frequency and sizes of these zones, offering insights into the textural properties of the image. In simpler terms, the GLSZM helps us understand how different shades of gray are arranged in the image and how large or small these regions of specific gray levels are.

In our analysis, we've observed that there are a lot of features that have a strong correlation (value = 1); with this information we conclude that a lor of the features are redundant and in the furture could be removed. However in this project we are going to keep all the features seen above and see how they perform in our classification models. In theory, our conclusion will be that they not affect the performance of the models.


Rigth now we are going to analyze the correlation between the features and the malignancy label.

In [ ]:
correlation_threshold = 0.35

correlation_with_malignancy = df.corr()['Malignancy']
high_correlation_features = correlation_with_malignancy[correlation_with_malignancy.abs() >= correlation_threshold].index
print(high_correlation_features)

# Create a bar plot to visualize correlations
plt.figure(figsize=(15, 9))
sns.barplot(x=correlation_with_malignancy.index, y=correlation_with_malignancy)
plt.xticks(rotation=90)
plt.title('Correlation with Malignancy')
plt.xlabel('Features')
plt.ylabel('Correlation')
plt.show()

As we can see, there are some features that have a high correlation with the label, so we are going to use them in our classification models to see how it performs

## Classification <a id="Classification"></a>
[Back to index](#Index)

We created some functions to help us test every model.

In [ ]:
def heatmap(test,pred):
    cm = confusion_matrix(test, pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
def testing(test,pred):
    accuracy = accuracy_score(test, pred)
    report = classification_report(test, pred, zero_division=1)
    print("Accuracy:", accuracy)
    print("Classification Report:\n", report)
    heatmap(test,pred)

In [ ]:
def cross_validation(model, a, b, cv=10):
    scores = cross_val_score(model, a, b, cv=cv, scoring='accuracy')
    plt.figure(figsize=(8, 4))  # Create a separate figure
    plt.hist(scores)
    plt.title('Cross Validation average score: {}'.format(np.average(scores)))
    plt.show()
    return np.average(scores)  # Return the average accuracy score

In [ ]:
def average_score(model, a, b):
    model_accuracies = []
    model_precisions = []
    for repetition in range(100):
        x_train, x_test, Y_train, Y_test = train_test_split(a, b, test_size=0.3)

        model.fit(x_train, Y_train)
        Y_pred = model.predict(x_test)

        accuracy = accuracy_score(Y_test, Y_pred)
        precision = precision_score(Y_test, Y_pred)

        model_accuracies.append(accuracy)
        model_precisions.append(precision)
        

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # Create two subplots

    ax1.hist(model_accuracies, color='blue', alpha=0.7, label='Accuracy')
    ax1.set_title('Accuracy average score: {}'.format(np.average(model_accuracies)))

    ax2.hist(model_precisions, color='green', alpha=0.7, label='Precision')
    ax2.set_title('Precision average score: {}'.format(np.average(model_precisions)))

    plt.show()

    average_accuracy = np.average(model_accuracies)
    average_precision = np.average(model_precisions)

    return average_accuracy, average_precision


In [ ]:
def importance(model, a):
    impor=pd.DataFrame({'feature':a.columns,
                             'importance':np.round(model.feature_importances_, 3)})
    impor.sort_values('importance',ascending=False, inplace =True)
    return impor

After noticing that there are higher correlation features than other, we did a test with the Random Forest Classifier: we tried only using the features with 0.5 correlation.

In [ ]:
testing_df = df[high_correlation_features]
testing_X = df.iloc[:, :-1]
testing_y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(testing_X, testing_y, test_size=0.3)

In [ ]:
testing_rf_classifier = RandomForestClassifier(criterion="entropy", max_depth=30, min_samples_leaf=2, n_estimators=200)
testing_rf_classifier.fit(X_train, y_train)
y_pred = testing_rf_classifier.predict(X_test)

In [ ]:
testing(y_test, y_pred)

In [ ]:
average_score(testing_rf_classifier, testing_X, testing_y)
cross_validation(testing_rf_classifier, testing_X, testing_y)

After noticing that diference in the percentages isn't significant, we decided to keep all the features.
 
We tried with 0.35 and 0.5 correlation values and the results were the same.

Now we are going to test the models and see which one performs better. With each model we are going to test its accuracy, precision and cross validation score. While doint it, we are going to save each models test and then compare it after, to see which one performs better.

For the models hyper-paramenters we used the default ones first. Then after completing all the code for all the models we did some parameter tuning and we verified that the change was almost not significant, still, we used the best parameters for each model.
 
The code for the parameter tuning is in the end of the notebook.

On the models that permit it, we show the importance of each feature.

In [ ]:
model_scores = []
X = df.iloc[:, :-1]  # Features (all columns except the last one)
y = df.iloc[:, -1]   # Target (the last column)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Decision Tree Classifier <a id="Decision-Tree-Classifier"></a>

In [ ]:
tree=DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=1, min_samples_split=10) 
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

In [ ]:
testing(y_test,y_pred)

In [ ]:
tree_accuracy,tree_precision = average_score(tree,X,y)
tree_cross = cross_validation(tree,X,y)
model_scores.append(("Decision Tree Classifier",tree_accuracy,tree_precision,tree_cross))

In [ ]:
importance(tree,X_train)

### Random Forest Classifier <a id="Random-Forest-Classifier"></a>

In [ ]:
rf_classifier = RandomForestClassifier(criterion="entropy", max_depth=30, min_samples_leaf=2, n_estimators=200)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

In [ ]:
testing(y_test,y_pred)

In [ ]:
rf_accuracy, rf_precision = average_score(rf_classifier,X,y)
rf_cross = cross_validation(rf_classifier,X,y)
model_scores.append(("Random Forest Classifier",rf_accuracy,rf_precision,rf_cross))

In [ ]:
importance(rf_classifier,X_train)

### Gradient Boosting Classifier <a id="Gradient-Boosting-Classifier"></a>

In [ ]:
GB = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_depth=6)
GB.fit(X_train, y_train)
y_pred = GB.predict(X_test)

In [ ]:
testing(y_test,y_pred)

In [ ]:
gb_accuracy, gb_precision = average_score(GB,X,y)
gb_cross = cross_validation(GB,X,y)
model_scores.append(("Gradient Boosting Classifier",gb_accuracy, gb_precision,gb_cross))

In [ ]:
importance(GB,X_train)

### K Neighbors Classifier <a id="Knn"></a>

Testing to see which number of neighbors is the best

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
y_pred = knn.predict(np.array(X_test))
score = accuracy_score(y_test, y_pred)
best_score = (1, score)
for i in range(2, 101):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(np.array(X_test))
    score = accuracy_score(y_test, y_pred)
    if score > best_score[1]:  # Compare with the accuracy score, not the tuple
        best_score = (i, score)

print("Best accuracy is " + str(best_score[1]) + " with " + str(best_score[0]) + " neighbors")

In [ ]:
knn = KNeighborsClassifier(n_neighbors=best_score[0])
knn.fit(X_train, y_train)
y_pred = knn.predict(np.array(X_test))

In [ ]:
testing(y_test,y_pred)

In [ ]:
average_score(knn,np.array(X),y)
cross_validation(knn,np.array(X),y)

### Support Vector Machines <a id="Support-Vector-Machines"></a>

In [ ]:
svclassifier = SVC(C=10, gamma='scale', kernel='rbf')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)

In [ ]:
testing(y_test,y_pred)

In [ ]:
svm_accuracy, svm_precision = average_score(svclassifier,X,y)
svm_cross = cross_validation(svclassifier,X,y)
model_scores.append(("Support Vector Machines",svm_accuracy, svm_precision,svm_cross))

### Naive Bayes <a id="Naive-Bayes"></a>

### GaussianNB <a id="GaussianNB"></a>

In [ ]:
gauss = GaussianNB()
gauss.fit(X_train, y_train)
y_pred = gauss.predict(X_test)

In [ ]:
testing(y_test,y_pred)

In [ ]:
gnb_accuracy, gnb_precision = average_score(gauss,X,y)
gnb_cross = cross_validation(gauss,X,y)
model_scores.append(("Gaussian Naive Bayes",gnb_accuracy, gnb_precision,gnb_cross))

### MultinomialNB <a id="MultinomialNB"></a>

In [ ]:
mult = MultinomialNB(alpha=0.5)
mult.fit(X_train, y_train)
y_pred = mult.predict(X_test)

In [ ]:
testing(y_test,y_pred)

In [ ]:
mnb_accuracy, mnb_precision = average_score(mult,X,y)
mnb_cross = cross_validation(mult,X,y)
model_scores.append(("Multinomial Naive Bayes",mnb_accuracy, mnb_precision,mnb_cross))

### Comparing the models

Here we compare all the models test scores:

In [ ]:
model_names, accuracies, precisions, cross_scores = zip(*model_scores)

x = np.arange(len(model_names))  # X-axis positions

bar_width = 0.3  # Increased bar width
bar_positions = np.arange(len(model_names))

plt.figure(figsize=(15, 6))

plt.bar(bar_positions - bar_width, accuracies, width=bar_width, label='Accuracy', color='b', alpha=0.7)
plt.bar(bar_positions, precisions, width=bar_width, label='Precision', color='g', alpha=0.7)
plt.bar(bar_positions + bar_width, cross_scores, width=bar_width, label='Cross-Validation', color='r', alpha=0.7)

plt.xlabel('Models')
plt.ylabel('Scores')
plt.title('Model Comparison')
plt.legend()
plt.xticks(bar_positions, model_names, rotation=45)

# Adding text labels for the actual percentages with separation
for i in range(len(model_names)):
    plt.text(bar_positions[i] - bar_width, accuracies[i] + 0.02, f'{accuracies[i]:.2%}', ha='center')
    plt.text(bar_positions[i], precisions[i] + 0.02, f'{precisions[i]:.2%}', ha='center')
    plt.text(bar_positions[i] + bar_width, cross_scores[i] + 0.02, f'{cross_scores[i]:.2%}', ha='center')

plt.tight_layout()
plt.show()

## Parameter Tuning <a id="Parameter-Tuning"></a>

Here are the parameter grids for the models to see which parameters are the best for each model.

In [ ]:
param_grid_decision_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
param_grid_random_forest = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
param_grid_gradient_boost = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 4, 5, 6]
}

For the K Neighbors Classifier we are going to use the best number of neighbors we found before.

In [ ]:
param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

For the Naive Bayes models there are no hyperparameters to tune for Gaussian.

In [ ]:
param_grid_multinomial_nb = {
    'alpha': [0.1, 0.5, 1.0, 2.0]
}

In [ ]:
models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boost': GradientBoostingClassifier(),
    'SVC': SVC(),
    'Gaussian Naive Bayes': GaussianNB(),
    'Multinomial Naive Bayes': MultinomialNB()
}

param_grids = {
    'Decision Tree': param_grid_decision_tree,
    'Random Forest': param_grid_random_forest,
    'Gradient Boost': param_grid_gradient_boost,
    'SVC': param_grid_svc,
    'Gaussian Naive Bayes': {},  # No hyperparameters for Gaussian Naive Bayes
    'Multinomial Naive Bayes': param_grid_multinomial_nb
}

In [ ]:
def find_best_model_parameters(classifier, parameters_grid, a, b):
    grid_search = GridSearchCV(estimator=classifier, param_grid=parameters_grid, cv=5, scoring='accuracy')
    grid_search.fit(a, b)

    return grid_search.best_params_, grid_search.best_score_

In [ ]:
best_params_and_scores = {}

for model_name, model in models.items():
    param_grid = param_grids[model_name]

    if not param_grid:
        continue

    best_params, best_score = find_best_model_parameters(model, param_grid, X, y)
    best_params_and_scores[model_name] = {'Best Parameters': best_params, 'Best Score': best_score}

# Print the results
for model_name, result in best_params_and_scores.items():
    print(f'{model_name}:')
    print(f'Best Parameters: {result["Best Parameters"]}')
    print(f'Best Score: {result["Best Score"]}\n')

![img.png](img.png)

We leave here a print of the before of the parameters of the models. The after you can check in the code above:

![before.png](before.png)

## Conclusion <a id="Conclusion"></a>
[Back to index](#Index)

Conclusioning the conlusion